In [3]:
import networkx as nx
from collections import defaultdict
from csv import DictReader
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_mutual_info_score, homogeneity_score
import random
import time
from igraph import *
import numpy as np
from datetime import datetime
from pathlib import Path
from scipy.stats import entropy
import tqdm

### import graph data

In [4]:
p = Path("./data/edgeData").iterdir()
files = [x for x in p if x.is_file() and "2017" not in x.stem and "2016" not in x.stem]
files

[PosixPath('data/edgeData/edges_2012_2.csv'),
 PosixPath('data/edgeData/edges_2011_2_2012_2.csv'),
 PosixPath('data/edgeData/edges_2014_1.csv'),
 PosixPath('data/edgeData/edges_2013_2.csv'),
 PosixPath('data/edgeData/edges_2011_2_2012_1.csv'),
 PosixPath('data/edgeData/edges_2011_2_2015_1.csv'),
 PosixPath('data/edgeData/edges_2011_2_2013_2.csv'),
 PosixPath('data/edgeData/edges_2011_2_2015_2.csv'),
 PosixPath('data/edgeData/edges_2011_2.csv'),
 PosixPath('data/edgeData/edges_2013_1.csv'),
 PosixPath('data/edgeData/edges_2011_2_2014_1.csv'),
 PosixPath('data/edgeData/edges_2014_2.csv'),
 PosixPath('data/edgeData/edges_2012_1.csv'),
 PosixPath('data/edgeData/edges_2011_2_2013_1.csv'),
 PosixPath('data/edgeData/edges_2015_1.csv'),
 PosixPath('data/edgeData/edges_2011_2_2014_2.csv'),
 PosixPath('data/edgeData/edges_2015_2.csv')]

In [ ]:
p = Path("data/edgeData").glob('**/*')
files = [x for x in p if x.is_file() and "2017" not in x.stem and "2016" not in x.stem]

files

In [5]:
file_name = Path("data/edgeData/edges_2012_2.csv")

df = pd.read_csv(file_name)
df["sum"] = df[["h0", "h1", "h2", "h3", "h4", "h5", "h6"]].sum(axis=1)
print(len(df))
df.head()

1239850


pair_txt  \
0  ['14MoQRwiUSb3kMhHqRLzv4xXGi52BeSsZA', '1CSRWq...   
1  ['1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V', '1P528B...   
2  ['1BNwjNsTLTep6phNTsZYwvnMdPEugWK4ok', '1CSRWq...   
3  ['1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V', '1DP2QQ...   
4  ['1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V', '1GWLtg...   

                                node1                               node2  h0  \
0  14MoQRwiUSb3kMhHqRLzv4xXGi52BeSsZA  1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V   2   
1  1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V  1P528BvucdDr3BjfTXVBHo5AXJNFrssouS   2   
2  1BNwjNsTLTep6phNTsZYwvnMdPEugWK4ok  1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V   2   
3  1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V  1DP2QQq7GMSSAW7hUaZC6QeZxbiAqbYjMH   2   
4  1CSRWqt9PtrRavL15ZnfHTx4BzdWBQJf3V  1GWLtgU9WUcdmuu2L7AnQZKQd4vx1w8KzJ   2   

   h1  h2  h3  h4  h5  h6  sum  
0   0   0   0   0   0   0    2  
1   0   0   0   0   0   0    2  
2   0   0   0   0   0   0    2  
3   0   0   0   0   0   0    2  
4   0   0   0   0   0   0    2

### import ground truth

In [ ]:
df_gt = pd.read_csv('data/ground_truth_id.csv.zip')
print(df_gt.shape)

df_gt = df_gt.dropna(axis=0, how='any', subset=["address", "entity"])
print(df_gt.shape)

df_gt = df_gt.drop_duplicates(subset="address", keep=False)
print(df_gt.shape)

entity_counts = df_gt["entity"].value_counts()
rare_entities = entity_counts[entity_counts < 10].index
df_gt = df_gt.loc[~df_gt["entity"].isin(rare_entities), :]
print(df_gt.shape)

df_gt.head()

In [ ]:
gt_addr = set(df_gt["address"])
sample_addr = set(df["node1"]).union(set(df["node2"]))
sample_known_addr = sample_addr.intersection(gt_addr)

print('ground truth addresses',len(gt_addr))
print('sample addressses:',len(sample_addr))
print('ground truth addresses in sample',len(sample_known_addr))

In [ ]:
df_gt_known = df_gt.loc[df_gt["address"].isin(sample_addr), ['address', 'entity']]

known_entities = set(df_gt_known['entity'])

# known_entity_addr_dict = {idx: set(df_gt_known.loc[df_gt_known["entity"] == e, "address"]) for idx, e in enumerate(known_entities)}

known_addr_entity_dict_list = [{a: idx for a in set(df_gt_known.loc[df_gt_known["entity"] == e, "address"])} for idx, e in enumerate(known_entities)]
known_addr_entity_dict = {}
[known_addr_entity_dict.update(d) for d in known_addr_entity_dict_list]

known_entity_counts = df_gt_known['entity'].value_counts().rename("count").to_frame()

print('ground truth entities in sample:',len(known_entities))
known_entity_counts

In [ ]:
known_entity_counts["file"] = file_name.stem
known_entity_counts["n_nodes_graph"] = len(sample_addr)
known_entity_counts["n_edges_graph"] = len(df)

now = datetime.now()
date_str = now.strftime("%Y%m%d_%H%M")

file_name_output = f"known_entity_counts_{file_name.stem}_res_{date_str}.csv"
output_path = Path(f"data/res/{file_name_output}")

known_entity_counts.to_csv(output_path, index=True)

In [ ]:
def get_labels(cs_addr, known_addr_entity_dict):
    res = {}
    for idx, c in enumerate(cs_addr):
        for a in c:
            if a in sample_known_addr:
                res[a] = idx
                
    labels_true = [] 
    labels_pred = []

    for a in res.keys():
        labels_true.append(known_addr_entity_dict[a])
        labels_pred.append(res[a])
    
    return labels_true, labels_pred

In [6]:
edge_tuples = df[["node1", "node2", "sum"]].itertuples(index=False)
g = Graph.TupleList(edge_tuples, directed=False, weights=True)
g_weights = g.es["weight"]
print(g.is_directed())
summary(g)

False
IGRAPH UNW- 5054 1239850 -- 
+ attr: name (v), weight (e)


In [ ]:
def get_initial_and_fixed(sample_size):
    known_idxs = [idx for idx, addr in enumerate(g.vs["name"]) if addr in sample_known_addr]

    initial = np.zeros(len(g.vs), dtype=np.int)
    
    rng = np.random.RandomState(32512163)
    
    sample_known_idxs = rng.choice(known_idxs, size=sample_size, replace=False) # TODO: set seed
    non_sample_idxs = np.setdiff1d(range(len(g.vs)), sample_known_idxs)

    for idx in sample_known_idxs:
        initial[idx] = known_addr_entity_dict[g.vs[idx]["name"]]

    
    available_labels = np.setdiff1d(range(len(g.vs)), initial[sample_known_idxs])[:len(g.vs) - sample_size]
    
    initial[non_sample_idxs] = available_labels
    
    fixed = np.zeros(len(g.vs), dtype=np.int)
    fixed[sample_known_idxs] = 1
    fixed = fixed.astype(bool)
    
    return initial, fixed

In [ ]:
print(len(g.vs))
print(len(sample_known_addr))
print()

props = [0, 0.1] + list(np.geomspace(start=0.01, stop=0.4, num=15)) # np.arange(0, 1, 0.05)
props = sorted(props)
print(props)
print()

sizes = [int(p * len(g.vs)) for p in props if int(p * len(g.vs)) <= len(sample_known_addr)]
print(sizes)

In [ ]:
for rnd_iter in tqdm.tqdm(range(100)):
    
    if rnd_iter > 0:
        g.rewire(n=4*len(g.es), mode="simple")
        g.es["weight"] = g_weights
        
    res_cols = ["file", "n_nodes_graph", "n_edges_graph", "prop_graph", "prop_known", 
                "n_clusters", "cluster_sizes", "ami", "homog", "mod", 
                "entropy_vs_cluster_entropy", 
                "entropy_vs_cluster_sizes_known_addr", 
                "entropy_vs_cluster_n_entities", 
                "entropy_vs_cluster_sizes_labels_true", 
                "entropy_vs_entity_entropy", 
                "entropy_vs_entity_abs_size",
                "entropy_vs_entity_rel_size"]
    
    res = {c: [] for c in res_cols}

    for i, size in enumerate(sizes):
        initial, fixed = get_initial_and_fixed(size)
        cs = g.community_label_propagation(weights="weight", initial=initial, fixed=fixed)

        cs_addr = sorted([g.vs.select(c)["name"] for c in cs], key=len)
        cs_sizes = [len(c) for c in cs_addr]

        labels_true, labels_pred = get_labels(cs_addr=cs_addr, known_addr_entity_dict=known_addr_entity_dict)

        ami = adjusted_mutual_info_score(labels_true=labels_true, labels_pred=labels_pred)
        homog = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)
        mod = cs.modularity

        res["file"].append(file_name.stem)
        res["n_nodes_graph"].append(len(sample_addr))
        res["n_edges_graph"].append(len(df))
        res["prop_graph"].append(props[i])
        res["prop_known"].append(size/len(sample_known_addr))
        res["n_clusters"].append(len(cs))
        res["cluster_sizes"].append(cs_sizes)
        res["ami"].append(ami)
        res["homog"].append(homog)
        res["mod"].append(mod)
        
        # print(f"{props[i]:.2%} \t {size/len(sample_known_addr):.2%}   \t ami: {ami:.2f} \t homog: {homog:.2f} \t {mod:.2f}")
        print(f"{size/len(sample_known_addr):.2%}  mod: {mod:.2f} ami: {ami:.2f} homog: {homog:.2f} num_clusters: {len(cs)}")
        #print(cs_sizes)
        #print("labels_true")
        #print(pd.Series(labels_true).value_counts())
        #print("labels_pred")
        #print(pd.Series(labels_pred).value_counts())
            
        entropy_vs_entity_entropy = []
        entropy_vs_entity_abs_size = []
        entropy_vs_entity_rel_size = []
        
        entity_cluster_distribution_dict = defaultdict(lambda: defaultdict(int))
        for entity, cluster in zip(labels_true, labels_pred):
            entity_cluster_distribution_dict[entity][cluster] += 1
            
        for entity, cluster_dist in entity_cluster_distribution_dict.items():
            e = entropy(list(cluster_dist.values()), base=2)
            entropy_vs_entity_entropy.append(e)
            entropy_vs_entity_abs_size.append(len(cluster_dist))
            entropy_vs_entity_rel_size.append(len(cluster_dist) / len(sample_known_addr))
        
        res["entropy_vs_entity_entropy"].append(entropy_vs_entity_entropy)
        res["entropy_vs_entity_abs_size"].append(entropy_vs_entity_abs_size)
        res["entropy_vs_entity_rel_size"].append(entropy_vs_entity_rel_size)
        
        n_entities = []
        entropys = []
        cluster_sizes_known_addr = []
        cluster_sizes_labels_true = []
        for c in cs_addr:
            labels_true, labels_pred = get_labels(cs_addr=[c], known_addr_entity_dict=known_addr_entity_dict)

            if len(labels_pred) == 0:
                continue
    
            _, counts = np.unique(labels_true, return_counts=True)

            e = entropy(counts, base=2)
            entropys.append(e)

            n_entity = len(np.unique(labels_true))
            n_entities.append(n_entity)

            cluster_sizes_known_addr.append(len(labels_true))
            cluster_sizes_labels_true.append(len(c))

        res["entropy_vs_cluster_n_entities"].append(n_entities)
        res["entropy_vs_cluster_entropy"].append(entropys)
        res["entropy_vs_cluster_sizes_known_addr"].append(cluster_sizes_known_addr)
        res["entropy_vs_cluster_sizes_labels_true"].append(cluster_sizes_labels_true)


In [ ]:
df_res = pd.DataFrame(res)
print(df_res.info())
df_res

In [ ]:
now = datetime.now()
date_str = now.strftime("%Y%m%d_%H%M")

file_name_output = f"{file_name.stem}_res_{date_str}.csv"
output_path = Path(f"data/res/{file_name_output}")

df_res.to_csv(output_path, index=False)

In [ ]:
import networkx as nx
from collections import defaultdict
from csv import DictReader
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_mutual_info_score, homogeneity_score
import random
import time
from igraph import *
import numpy as np
from datetime import datetime
from pathlib import Path
from scipy.stats import entropy
import tqdm

p = Path("./data/edgeData").iterdir()
files = [x for x in p if x.is_file() and "edges_2015_1" in x.stem and "2017" not in x.stem and "2016" not in x.stem]

for file_name in files:
	
	print(file_name)
	
	df = pd.read_csv(file_name)
	df["sum"] = df[["h0", "h1", "h2", "h3", "h4", "h5", "h6"]].sum(axis=1)
	# print(len(df))

	df_gt = pd.read_csv('data/ground_truth_id.csv.zip')
	# print(df_gt.shape)

	df_gt = df_gt.dropna(axis=0, how='any', subset=["address", "entity"])
	# print(df_gt.shape)

	df_gt = df_gt.drop_duplicates(subset="address", keep=False)
	# print(df_gt.shape)

	entity_counts = df_gt["entity"].value_counts()
	rare_entities = entity_counts[entity_counts < 10].index
	df_gt = df_gt.loc[~df_gt["entity"].isin(rare_entities), :]
	# print(df_gt.shape)

	gt_addr = set(df_gt["address"])
	sample_addr = set(df["node1"]).union(set(df["node2"]))
	sample_known_addr = sample_addr.intersection(gt_addr)

	# print('ground truth addresses',len(gt_addr))
	# print('sample addressses:',len(sample_addr))
	# print('ground truth addresses in sample',len(sample_known_addr))

	df_gt_known = df_gt.loc[df_gt["address"].isin(sample_addr), ['address', 'entity']]

	known_entities = set(df_gt_known['entity'])

	# known_entity_addr_dict = {idx: set(df_gt_known.loc[df_gt_known["entity"] == e, "address"]) for idx, e in enumerate(known_entities)}

	known_addr_entity_dict_list = [{a: idx for a in set(df_gt_known.loc[df_gt_known["entity"] == e, "address"])} for idx, e in enumerate(known_entities)]
	known_addr_entity_dict = {}
	[known_addr_entity_dict.update(d) for d in known_addr_entity_dict_list]

	known_entity_counts = df_gt_known['entity'].value_counts().rename("count").to_frame()

	# print('ground truth entities in sample:',len(known_entities))

	known_entity_counts["file"] = file_name.stem
	known_entity_counts["n_nodes_graph"] = len(sample_addr)
	known_entity_counts["n_edges_graph"] = len(df)

	now = datetime.now()
	date_str = now.strftime("%Y%m%d_%H%M")

	file_name_output = f"known_entity_counts_{file_name.stem}_res_{date_str}.csv"
	output_path = Path(f"data/res/{file_name_output}")

	known_entity_counts.to_csv(output_path, index=True)

	def get_labels(cs_addr, known_addr_entity_dict):
		res = {}
		for idx, c in enumerate(cs_addr):
			for a in c:
				if a in sample_known_addr:
					res[a] = idx
					
		labels_true = [] 
		labels_pred = []

		for a in res.keys():
			labels_true.append(known_addr_entity_dict[a])
			labels_pred.append(res[a])
		
		return labels_true, labels_pred
		
	edge_tuples = df[["node1", "node2", "sum"]].itertuples(index=False)
	g = Graph.TupleList(edge_tuples, directed=False, weights=True)
	g_weights = g.es["weight"]

	def get_initial_and_fixed(sample_size):
		known_idxs = [idx for idx, addr in enumerate(g.vs["name"]) if addr in sample_known_addr]

		initial = np.zeros(len(g.vs), dtype=np.int)
		
		rng = np.random.RandomState(32512163)
		
		sample_known_idxs = rng.choice(known_idxs, size=sample_size, replace=False) # TODO: set seed
		non_sample_idxs = np.setdiff1d(range(len(g.vs)), sample_known_idxs)

		for idx in sample_known_idxs:
			initial[idx] = known_addr_entity_dict[g.vs[idx]["name"]]

		
		available_labels = np.setdiff1d(range(len(g.vs)), initial[sample_known_idxs])[:len(g.vs) - sample_size]
		
		initial[non_sample_idxs] = available_labels
		
		fixed = np.zeros(len(g.vs), dtype=np.int)
		fixed[sample_known_idxs] = 1
		fixed = fixed.astype(bool)
		
		return initial, fixed
		

	# print(len(g.vs))
	# print(len(sample_known_addr))
	# print()

	props = [0, 0.1] + list(np.geomspace(start=0.01, stop=0.4, num=15)) # np.arange(0, 1, 0.05)
	props = sorted(props)
	# print(props)
	# print()

	sizes = [int(p * len(g.vs)) for p in props if int(p * len(g.vs)) <= len(sample_known_addr)]
	# print(sizes)

	total_rnd_iter = 100
	for rnd_iter in range(total_rnd_iter):
		print(f"{rnd_iter + 1} / {total_rnd_iter}")
		
		if rnd_iter > 0:
			g.rewire(n=4*len(g.es), mode="simple")
			g.es["weight"] = g_weights
			print("randomized")
			
		res_cols = ["file", "n_nodes_graph", "n_edges_graph", "prop_graph", "prop_known", 
					"n_clusters", "cluster_sizes", "ami", "homog", "mod", 
					"entropy_vs_cluster_entropy", 
					"entropy_vs_cluster_sizes_known_addr", 
					"entropy_vs_cluster_n_entities", 
					"entropy_vs_cluster_sizes_labels_true", 
					"entropy_vs_entity_entropy", 
					"entropy_vs_entity_abs_size",
					"entropy_vs_entity_rel_size"]
		
		res = {c: [] for c in res_cols}

		for i, size in enumerate(sizes):
			initial, fixed = get_initial_and_fixed(size)
			cs = g.community_label_propagation(weights="weight", initial=initial, fixed=fixed)

			cs_addr = sorted([g.vs.select(c)["name"] for c in cs], key=len)
			cs_sizes = [len(c) for c in cs_addr]

			labels_true, labels_pred = get_labels(cs_addr=cs_addr, known_addr_entity_dict=known_addr_entity_dict)

			ami = adjusted_mutual_info_score(labels_true=labels_true, labels_pred=labels_pred)
			homog = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)
			mod = cs.modularity

			res["file"].append(file_name.stem)
			res["n_nodes_graph"].append(len(sample_addr))
			res["n_edges_graph"].append(len(df))
			res["prop_graph"].append(props[i])
			res["prop_known"].append(size/len(sample_known_addr))
			res["n_clusters"].append(len(cs))
			res["cluster_sizes"].append(cs_sizes)
			res["ami"].append(ami)
			res["homog"].append(homog)
			res["mod"].append(mod)
			
			print(f"{size/len(sample_known_addr):.2%}  mod: {mod:.2f} ami: {ami:.2f} homog: {homog:.2f} num_clusters: {len(cs)}")
			
			entropy_vs_entity_entropy = []
			entropy_vs_entity_abs_size = []
			entropy_vs_entity_rel_size = []
			
			entity_cluster_distribution_dict = defaultdict(lambda: defaultdict(int))
			for entity, cluster in zip(labels_true, labels_pred):
				entity_cluster_distribution_dict[entity][cluster] += 1
				
			for entity, cluster_dist in entity_cluster_distribution_dict.items():
				e = entropy(list(cluster_dist.values()), base=2)
				entropy_vs_entity_entropy.append(e)
				entropy_vs_entity_abs_size.append(len(cluster_dist))
				entropy_vs_entity_rel_size.append(len(cluster_dist) / len(sample_known_addr))
			
			res["entropy_vs_entity_entropy"].append(entropy_vs_entity_entropy)
			res["entropy_vs_entity_abs_size"].append(entropy_vs_entity_abs_size)
			res["entropy_vs_entity_rel_size"].append(entropy_vs_entity_rel_size)
			
			n_entities = []
			entropys = []
			cluster_sizes_known_addr = []
			cluster_sizes_labels_true = []
			for c in cs_addr:
				labels_true, labels_pred = get_labels(cs_addr=[c], known_addr_entity_dict=known_addr_entity_dict)

				if len(labels_pred) == 0:
					continue
		
				_, counts = np.unique(labels_true, return_counts=True)

				e = entropy(counts, base=2)
				entropys.append(e)

				n_entity = len(np.unique(labels_true))
				n_entities.append(n_entity)

				cluster_sizes_known_addr.append(len(labels_true))
				cluster_sizes_labels_true.append(len(c))

			res["entropy_vs_cluster_n_entities"].append(n_entities)
			res["entropy_vs_cluster_entropy"].append(entropys)
			res["entropy_vs_cluster_sizes_known_addr"].append(cluster_sizes_known_addr)
			res["entropy_vs_cluster_sizes_labels_true"].append(cluster_sizes_labels_true)

		df_res = pd.DataFrame(res)
		# print(df_res.info())

		now = datetime.now()
		date_str = now.strftime("%Y%m%d_%H%M_%s")

		file_name_output = f"{file_name.stem}_res_{date_str}.csv"
		output_path = Path(f"data/res/{file_name_output}")

		df_res.to_csv(output_path, index=False)


data/edgeData/edges_2015_1.csv
1 / 100
0.00%  mod: 0.85 ami: 0.41 homog: 0.83 num_clusters: 1404
1.94%  mod: 0.73 ami: 0.59 homog: 0.82 num_clusters: 1126
2.52%  mod: 0.72 ami: 0.62 homog: 0.82 num_clusters: 1109
3.28%  mod: 0.71 ami: 0.65 homog: 0.81 num_clusters: 989
4.27%  mod: 0.71 ami: 0.66 homog: 0.82 num_clusters: 947
5.56%  mod: 0.71 ami: 0.69 homog: 0.81 num_clusters: 887
7.23%  mod: 0.71 ami: 0.72 homog: 0.81 num_clusters: 783
9.41%  mod: 0.70 ami: 0.75 homog: 0.82 num_clusters: 720
12.25%  mod: 0.71 ami: 0.77 homog: 0.82 num_clusters: 635
15.94%  mod: 0.71 ami: 0.79 homog: 0.82 num_clusters: 554
19.37%  mod: 0.71 ami: 0.80 homog: 0.83 num_clusters: 485
20.75%  mod: 0.70 ami: 0.80 homog: 0.83 num_clusters: 479
27.00%  mod: 0.70 ami: 0.82 homog: 0.84 num_clusters: 428
35.14%  mod: 0.70 ami: 0.84 homog: 0.85 num_clusters: 352
45.74%  mod: 0.70 ami: 0.87 homog: 0.87 num_clusters: 328
59.53%  mod: 0.69 ami: 0.90 homog: 0.90 num_clusters: 301
77.47%  mod: 0.69 ami: 0.94 homog: 0.9